This code was constructed referencing the work of [Nadin Tamer](https://www.kaggle.com/nadintamer), a user here on Kaggle that has done analysis on top Spotify tracks for a few years now. 

Additionally, this is my first venture into using Python. I'm completely open to constructive criticism; if you have a suggestion or a comment, please leave it below!

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing Necessary Libraries

For this analysis, let's use numpy, pandas, matplotlib.pyplot, and seaborn. 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


Let's read in the data and get an idea of what our data looks like.

In [ ]:

music = pd.read_csv("../input/top-spotify-tracks-of-2019/top2019.csv")
music.head() #Displaying the first 5 entries


In [ ]:
music.describe(include="all")

#NaN == Not a Number - indicates qualitative variables.

#Note that this table just provides some descriptive statistics so we can continue getting an idea of what each value means,
#and what values we should be expecting to find.



# Analysis

In [ ]:
quant = music.drop(['id','name', 'artists'], axis=1) #Creating a table for the quantitative data
smallQuant = quant.drop(['tempo','duration_ms','key','loudness','time_signature'], axis = 1) #Table for data 0 <= x <= 1

sns.set_palette('rocket') #Style choice

In [ ]:
#Let's make our first bar chart. This should give us an idea of how 'danceable' a song is on average, or what the energy of a popular song is on average. 

smallQuant.mean().plot.bar() #Creates a bar chart of the mean values of all variables with values 0 < x < 1.
plt.title('Mean Values of Audio Qualities')
plt.show()

This gives us a fantastic impression of what qualities, in general, constitute a popular song on Spotify. Note that all of these values are on a scale from 0-1, e.g. a song that is easier to dance to  will have a danceability score closer to 1. We can see that the database consists of songs that have an average of ~0.71 and ~0.58 danceability and energy. We also notice that speechiness and instrumentalness are quite low, meaning that songs with vocals are more popular than instrumentals, but songs that are made up of vocals without backing instruments also do not take up much space on the chart. This shouldn't come as a surprise to anyone who listens to top 40 radio, as there isn't very much spoken-word poetry on the list.

# Energy and Danceability

In [ ]:
print("Mean energy levels:", music['energy'].mean())
sns.distplot(music['energy']) #Distribution plot of energy
plt.show()

print("Mean danceability value:", music['danceability'].mean())
sns.distplot(music['danceability'])

plt.show()

We can see that, in general, popular songs have energy levels > 0.50 and danceability levels > 0.60. 

Both variables appear to be normally distributed and left-skewed.

**Inference:** Popular songs tend to have higher energy. Popular songs tend to have higher danceability.

**Questions Raised:** Why are high-energy and danceable songs more popular? Is this because of the positive emotions and endorphins they drum up? Do younger people tend to like higher energy songs, and do younger people make up a large-enough percentage of Spotify's userbase to influence this? - Yes, over half of Spotify's userbase is under the age of 35. 
 

**Inaccurate Inferences:** ~~High enery songs are more popular. High danceability songs are more popular.~~ These inferences are imprecise - There exist songs that are high-energy but are not popular (Pick your favorite Jazz track and you'll probably find that it's not in the top 40). The same goes for danceability - Swing songs would likely score high on danceability, but are certainly not present in today's top songs.

In [ ]:
#Information on Qualitative data - we need to map some values to our qualitative data to create plots. 
#We'll do this by scale.

note_mapping = {0.0: 'C', 1.0: 'C♯,D♭', 2.0: 'D', 3.0: 'D♯,E♭', 4.0: 'E', 5.0: 'F', 6.0: 'F♯,G♭', 7.0: 'G', 8.0: 'G♯,A♭', 9.0: 'A', 10.0: 'A♯,B♭', 11.0: 'B'}
music['key'] = music['key'].map(note_mapping) #Maps each note to an integer

sns.countplot(x = 'key', data = music, order =music['key'].value_counts().index)
sns.set_palette('mako')
plt.title("Count of Keys")
plt.show()


We can see that C♯,D♭ and F are the keys that occur most frequently. Note that C♯ and D♭ are enharmonic equivalents - they produce the same pitch.

**Questions Raised** Why are these keys the most popular? C# is most commonly used in rock music, which is largely not present in the top tracks. 

# Heatmap

In [ ]:
plt.figure(figsize = (20,8))
sns.heatmap(quant.corr(), cmap="YlGnBu", annot=True)
plt.show()

This heatmap shows us the correlation between each variable - the closer the value to 1, the stronger the correlation. For example, energy and loudness have a correlation coefficient of 0.81 - the data suggests a strong correlation between the loudness of a song and the energy of a song. Common sense check; does this match up with your experience of music? 

Another common sense check - does it make sense that acousticness has a negative correlation of > 0.40 with danceability, energy, and loudness? Absolutely! As a song gets 'more acoustic' it's less likely that the song is written such that it's meant to be played in a club with everyone dancing. As a song gets 'more acoustic', the energy and loudness will be lower to match the mellow energy of a typical acoustic track.

A particularly interesting point comes at the instrumentalness-speechiness correlation of 0.44. Shouldn't it be the case that a highly instrumental track is *not* wordy? A possible explanation to explore here is that hip-hop music is counted as both highly instrumental and speechy. Since hip-hop accounts for a large portion of the top tracks, this genre alone could drive this correlation. Let's confirm!

# Hip-Hop